In [1]:
import numpy as np
import random

# Old way

In [33]:
from Gillespie_SEIR_function_diffnetwork import single_model_run_SEIR

n = 20
final_timepoint = 10
total_pop = 10**4
b = 0.8
# gamma: probability of becoming infectious
g = 1
# mu: probability of recovery
m = 1/5
# lambda: probability of losing immunity
l = 2/365
# p: probability of moving
p = 0.01

t,x,em = single_model_run_SEIR(
    final_timepoint,
    total_pop,
    n,
    [b,g,m,l,p],
    edges_mat
)

1.005255733775445
2.019096325230236
3.02131744128067
4.026345287669574
5.026447782424373
6.030270505061512
7.030282875072196
8.03289599502446
9.03629799709218


# Splitting it up into functions (although I don't really understand what they do)

In [34]:
def create_edge_matrix(n_nodes, min_edges):
    n = n_nodes
    max_edges = np.floor(np.sqrt(n)) # maximum number of edges for a node
    max_edges = max_edges.astype(int)
    # get poisson distribution of degrees
    pdf = [x**(-3) for x in range(min_edges,max_edges+1)]
    cdf = np.cumsum(pdf)
    n_edges_per_node = np.random.uniform(0,cdf[-1],n)
    list_of_stubs = []
    # get number of edges per node, save and save in list of stubs
    for i in range(n_nodes):
        c_edges = 0
        # find minimum number of degrees for which U(0,1) is more than poisson prob.
        while n_edges_per_node[i] > cdf[c_edges]:
            c_edges += 1
        n_edges_per_node[i] = c_edges + min_edges # save number of edges
        list_of_stubs.extend([i]*n_edges_per_node[i].astype(int)) # save number of stubs for network building
    if len(list_of_stubs)%2 != 0: #if the number of edges is not even, we need to fix that
        r_int = random.randint(0,n-1)
        while list_of_stubs.count(list_of_stubs[r_int]) <= min_edges: # check that we don't decrease degree belwo minimum
            r_int = random.randint(0,n-1)
        list_of_stubs.remove(r_int)
    edges_mat = np.zeros((n,n)) # initiate edges matrix
    list_of_edges = [] # initiate list of all edges in network
    while len(list_of_stubs)>0:
        if (len(list_of_stubs) == 2) & (list_of_stubs[0] == list_of_stubs[1]): # cannot connect to own node
            # break up previously made edge, and make two new ones
            edges_mat[last_edge[0]][last_edge[1]] = 0
            edges_mat[last_edge[1]][last_edge[0]] = 0
            edges_mat[last_edge[0]][list_of_stubs[0]] = 1
            edges_mat[last_edge[1]][list_of_stubs[1]] = 1
            edges_mat[list_of_stubs[0]][last_edge[0]]= 1
            edges_mat[list_of_stubs[1]][last_edge[1]] = 1
            break
        edge = random.sample(list_of_stubs,2) # create edge from two stubs
        if (edge[0] != edge[1]) & ~(edge in list_of_edges) & ~([edge[1], edge[0]] in list_of_edges): # check if not connecting to self and edge doesn't already exist
            edges_mat[edge[0]][edge[1]] = 1 # connect nodes in edges matrix
            edges_mat[edge[1]][edge[0]] = 1
            list_of_stubs.remove(edge[0]) # remove stubs from list
            list_of_stubs.remove(edge[1])
            last_edge = edge
            list_of_edges.append(edge)
    return edges_mat

def create_stochiometric_matrix(
    n_compartments, n_reactions_per_compartment,
    edges_mat):
    n = edges_mat.shape[0]
    c = n_compartments
    n_rxn = n_reactions_per_compartment
    rxn = np.zeros((n*(n_rxn+n*c), n*c))
    for i in range(n):
        # compartment reactions
        StoE = np.repeat(0,n*c)
        StoE[i*c] = -1
        StoE[i*c+1] = 1
        rxn[i*(n_rxn+(n*c))] = StoE
        EtoI = np.repeat(0,n*c)
        EtoI[i*c+1] = -1
        EtoI[i*c+2] = 1
        rxn[i*(n_rxn+(n*c))+1] = EtoI
        ItoR = np.repeat(0,n*c)
        ItoR[i*c+2] = -1
        ItoR[i*c+3] = 1
        rxn[i*(n_rxn+(n*c))+2] = ItoR
        RtoS = np.repeat(0,n*c)
        RtoS[i*c+3] = -1
        RtoS[i*c] = 1
        rxn[i*(n_rxn+(n*c))+3] = RtoS
        # movement reactions
        #count = 0
        for j in range(n):
            if edges_mat[i][j] == 1:
                Sitoj = np.repeat(0,n*c)
                Sitoj[i*c] = -1
                Sitoj[j*c] = 1
                rxn[i*(n_rxn+(n*c)) + c + j*c] = Sitoj
                Eitoj = np.repeat(0,n*c)
                Eitoj[i*c + 1] = -1
                Eitoj[j*c + 1] = 1
                rxn[i*(n_rxn+(n*c)) + c + j*c + 1] = Eitoj
                Iitoj = np.repeat(0,n*c)
                Iitoj[i*c + 2] = -1
                Iitoj[j*c + 2] = 1
                rxn[i*(n_rxn+(n*c)) + c + j*c + 2] = Iitoj
                Ritoj = np.repeat(0,n*c)
                Ritoj[i*c + 3] = -1
                Ritoj[j*c + 3] = 1
                rxn[i*(n_rxn+(n*c)) + c + j*c + 3] = Ritoj
    return rxn

def calculate_propensities(n_nodes, n_compartments, n_edges_per_node, x, parameters):
    
    b = parameters.rate_infection
    g = parameters.rate_infectious
    m = parameters.rate_recovery
    l = parameters.rate_waning
    p = parameters.rate_moving
    
    n = n_nodes
    c = n_compartments
    prop = np.array([])
    for i in range(n):
        # patch 1: S to E, E to I, I to R, R to S, move 1 to 2 (for each comp), move 1 to 3 (for each comp)
        N = np.sum(x[i*c:(i+1)*c])
        if N == 0:
            pinfect = np.zeros(c)
            pmove = np.zeros(n*c)
        else:
            pinfect = np.array([(1-(1-b/N)**x[i*c+2])*x[i*c], g*x[i*c+1], m*x[i*c+2], l*x[i*c+3]])
            pmove = np.zeros(n*c)
            for j in range(n):
                if edges_mat[i][j] == 1:
                    first_index = j*c
                    pmove[first_index:first_index+c] = p/n_edges_per_node[i] *np.array([x[i*c], x[i*c+1], x[i*c+2], x[i*c+3]])
            pmove = np.array(pmove)
        p_temp = np.concatenate((pinfect, pmove))
        prop = np.concatenate((prop, p_temp))
    return prop


class Parameters():
    def __init__(self, rate_infection, rate_infectious,
                 rate_recovery, rate_waning,
                 rate_moving):
        self.rate_infection = rate_infection
        self.rate_infectious = rate_infectious
        self.rate_recovery = rate_recovery
        self.rate_waning = rate_waning
        self.rate_moving = rate_moving
        
def run_model(
    t_final,
    total_pop,
    parameters,
    edges_mat,
    n_compartments,
    n_reactions_within_compartments):
    
    c = n_compartments
    n_rxn = n_reactions_within_compartments
    n = edges_mat.shape[0]
    approx_total_number_individuals = total_pop
    avg_pop_size = approx_total_number_individuals / n
    x = np.repeat(0,n*c) # initialized compartments for all nodes
    x[0:-1:c] = np.rint(np.random.normal(1,0.5,n)*avg_pop_size) # initial all susceptible populations
    # infect random node
    init_node = random.randint(0,n-1)*c
    x[init_node:init_node+c] = [x[init_node]*0.95, 0, x[init_node]*0.05, 0] # initial infections in first node
    x[x < 0] = 0
    x_store = [x]
    t_store = [0]
    t = 0
    
    rand_to_generate = 10**6
    random_numbers = np.random.uniform(0,1,rand_to_generate)
    random_number_count = 0
    
    
    n_edges_per_node = [np.sum(x) for x in edges_mat]
    rxn = create_stochiometric_matrix(
    n_compartments, n_reactions_within_compartments,
    edges_mat)
    
    tracker = 0
    while t <= t_final:
        if tracker > 1:
            print(t)
            tracker = 0
        prop = calculate_propensities(n, c, n_edges_per_node, x, parameters)
        prop_cumsummed = np.cumsum(prop)
        
        if random_number_count + 1 >= rand_to_generate:
            random_numbers = np.random.uniform(0,1,rand_to_generate)
            random_number_count = 0
        
        # compute time step
        dt = - np.log(random_numbers[random_number_count])*(1/prop_cumsummed[-1])
        random_number_count += 1
        
        t = t + dt
        tracker += dt
        t_store.append(t)
        
        # choose reaction
        rn = random_numbers[random_number_count]
        random_number_count += 1
        
        prop_cumsummed = np.divide(prop_cumsummed,prop_cumsummed[-1])
        
        for count in range(len(prop)):
            if rn <= prop_cumsummed[count]:
                break
            
        # find reaction in list from above
        current_rxn = rxn[count]
        # add reaction to population vector
        x = [sum(temp) for temp in zip(x, current_rxn)]
        x_store.append(x)
        
    return t_store, np.array(x_store), edges_mat

In [35]:
edges_mat = create_edge_matrix(20, 2)
parameters = Parameters(
    rate_infection=0.8,
    rate_infectious=1,
    rate_recovery=1/5,
    rate_waning=2/365,
    rate_moving=0.01)
n_compartments = 4
n_reactions_within_compartments = 4

t,x,em = run_model(
    10,
    10**4,
    parameters,
    edges_mat,
    n_compartments,
    n_reactions_within_compartments)

1.0005929225208892
2.0022275218894094
3.003232426520132
4.00372773249195
5.006952787598654
6.012443497915912
7.018995429729193
8.019368358103554
9.02254453472492


Open question: can we use available software to make an Erdos-Renyi graph?

In [37]:
import networkx as nx

n = 10  # 10 nodes
m = 20  # 20 edges
seed = 20160  # seed random number generators for reproducibility

# Use seed for reproducibility
G = nx.gnm_random_graph(n, m, seed=seed)

# Makes adjacency matrix
A = nx.adjacency_matrix(G)
A[1,0]

1